# **RAG Evaluation**
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.

For an introduction to RAG, you can check [this other cookbook](rag_zephyr_langchain)!

RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system's performance!
So let's see how to evaluate our RAG system.

### Evaluating RAG performance

Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.

For our evaluation pipeline, we will need:
1. An evaluation dataset with question - answer couples (QA couples)
2. An evaluator to compute the accuracy of our system on the above evaluation dataset.

➡️ It turns out, we can use LLMs to help us all along the way!
1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

In [1]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille
!pip install -U langchain-text-splitters langchain-community langchain
!pip install -U langchain-huggingface
!pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.

In [2]:
# %reload_ext autoreload
# %autoreload 2

In [3]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth",None) #Hiển thị trọn vẹn nội dung văn bản

In [4]:
from huggingface_hub import notebook_login

notebook_login()

### **Load your knowledge base**

In [6]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [7]:
# Check dataset
print(ds)

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})


In [8]:
import pandas as pd

df_view = pd.DataFrame(ds.select(range(5)))
display(df_view)

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# **1. Build a synthetic dataset for evaluation**
We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.

Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### **1.1. Prepare source documents**

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(
        page_content=doc["text"],
        metadata={"source": doc["source"]}
    )
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

print(len(docs_processed))

  0%|          | 0/2647 [00:00<?, ?it/s]

28308


In [10]:
# Check (ds --> langchain_docs --> docs_processed)
import random
i = random.randint(1,1000)
print("\n===== DS =====")
print(ds[i])

print("\n===== LANGCHAIN DOCUMENT =====")
print(langchain_docs[i])

print("\n===== PROCESSED DOCUMENT (CHUNK) =====")
print(docs_processed[i])



===== DS =====
{'text': ' Archived Changes\n\n### Nov 22, 2021\n* A number of updated weights anew new model defs\n  * `eca_halonext26ts` - 79.5 @ 256\n  * `resnet50_gn` (new) - 80.1 @ 224, 81.3 @ 288\n  * `resnet50` - 80.7 @ 224, 80.9 @ 288 (trained at 176, not replacing current a1 weights as default since these don\'t scale as well to higher res, [weights](https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1h2_176-001a1197.pth))\n  * `resnext50_32x4d` - 81.1 @ 224, 82.0 @ 288\n  * `sebotnet33ts_256` (new) - 81.2 @ 224\n  * `lamhalobotnet50ts_256` - 81.5 @ 256\n  * `halonet50ts` - 81.7 @ 256\n  * `halo2botnet50ts_256` - 82.0 @ 256\n  * `resnet101` - 82.0 @ 224, 82.8 @ 288\n  * `resnetv2_101` (new) - 82.1 @ 224, 83.0 @ 288\n  * `resnet152` - 82.8 @ 224, 83.5 @ 288\n  * `regnetz_d8` (new) - 83.5 @ 256, 84.0 @ 320\n  * `regnetz_e8` (new) - 84.5 @ 256, 85.0 @ 320\n* `vit_base_patch8_224` (85.8 top-1) & `in21k` variant weights added thanks [Mart

### **1.2. Setup agents for question generation**

We use [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation because it it has excellent performance in leaderboards such as [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [11]:
from huggingface_hub import InferenceClient


# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"
repo_id = "meta-llama/Llama-3.2-3B-Instruct"

llm_client = InferenceClient(model=repo_id, timeout=120)


def call_llm(inference_client: InferenceClient, prompt: str):

    messages1 = [{"role": "user", "content": prompt}]

    response = inference_client.chat_completion(
        messages=messages1,
        max_tokens=200,
        temperature=0.1
    )
    return response.choices[0].message.content

# Run
try:
    result = call_llm(llm_client, "Việt Nam có bao nhiêu tỉnh/thành?")
    print(result)
except Exception as e:
    print(f"Lỗi: {e}")

Việt Nam hiện có 63 tỉnh, thành phố.


In [12]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [13]:

import random

idx = random.randint(0, len(docs_processed)-1)
sample_context = docs_processed[idx].page_content


print(f"--- ĐOẠN VĂN GỐC ---\n{sample_context}\n")
result = call_llm(llm_client, QA_generation_prompt.format(context=sample_context))
print(f"--- AI TỰ ĐẶT CÂU HỎI ---\n{result}")

--- ĐOẠN VĂN GỐC ---
Here's an example of how to use these attributes to create a Gradio app that does not lazy load and has an initial height of 0px.

```html
<gradio-app
	space="gradio/Echocardiogram-Segmentation"
	eager="true"
	initial_height="0px"
></gradio-app>
```

Here's another example of how to use the `render` event. An event listener is used to capture the `render` event and will call the `handleLoadComplete()` function once rendering is complete. 

```html
<script>
	function handleLoadComplete() {
		console.log("Embedded space has finished rendering");
	}

	const gradioApp = document.querySelector("gradio-app");
	gradioApp.addEventListener("render", handleLoadComplete);
</script>
```

--- AI TỰ ĐẶT CÂU HỎI ---
Factoid question: What is the default value of the `eager` attribute in the Gradio app tag?
Answer: true


Now let's generate our QA couples.
For this example, we generate only 5 QA couples and will load the rest from the Hub.

But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.

In [26]:
import random
import re

N_GENERATIONS = 5  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for idx in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    raw_output = call_llm(
        llm_client, QA_generation_prompt.format(context=idx.page_content)
    )

    try:

        question_match = re.search(r"(?:Question|Factoid question):\s*(.+?)\n", raw_output, re.IGNORECASE)
        answer_match = re.search(r"Answer:\s*(.+)", raw_output, re.IGNORECASE)

        if question_match and answer_match:
            question = question_match.group(1).split("Answer:")[0].strip()
            answer = answer_match.group(1).strip()

            if 10 < len(answer) < 500:
                outputs.append({
                    "context": idx.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": idx.metadata["source"],
                })
    except Exception as e:
        continue

Generating 5 QA couples...


  0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
# Check
print(outputs)
print(f"Số outputs được giữ lại sau khi lọc: {len(outputs)}")

[{'context': '*Large-scale NLP models have been shown to significantly improve the performance on language tasks with no signs of saturation. They also demonstrate amazing few-shot capabilities like that of human beings. This paper aims to explore large-scale models in computer vision. We tackle three major issues in training and application of large vision models, including training instability, resolution gaps between pre-training and fine-tuning, and hunger on labelled data. Three main techniques are proposed: 1) a residual-post-norm method combined with cosine attention to improve training stability; 2) A log-spaced continuous position bias method to effectively transfer models pre-trained using low-resolution images to downstream tasks with high-resolution inputs; 3) A self-supervised pre-training method, SimMIM, to reduce the needs of vast labeled images', 'question': 'What are the three major issues tackled in the training and application of large vision models?', 'answer': 'Tra

In [28]:

display(pd.DataFrame(outputs).head(5))

,context,question,answer,source_doc
0,"*Large-scale NLP models have been shown to significantly improve the performance on language tasks with no signs of saturation. They also demonstrate amazing few-shot capabilities like that of human beings. This paper aims to explore large-scale models in computer vision. We tackle three major issues in training and application of large vision models, including training instability, resolution gaps between pre-training and fine-tuning, and hunger on labelled data. Three main techniques are proposed: 1) a residual-post-norm method combined with cosine attention to improve training stability; 2) A log-spaced continuous position bias method to effectively transfer models pre-trained using low-resolution images to downstream tasks with high-resolution inputs; 3) A self-supervised pre-training method, SimMIM, to reduce the needs of vast labeled images",What are the three major issues tackled in the training and application of large vision models?,"Training instability, resolution gaps between pre-training and fine-tuning, and hunger on labelled data.",huggingface/transformers/blob/main/docs/source/en/model_doc/swinv2.md
1,"```bash\ngit lfs install\nmkdir data\ngit -C ""./data"" clone https://huggingface.co/datasets/codeparrot/codeparrot-clean-train\ngit -C ""./data"" clone https://huggingface.co/datasets/codeparrot/codeparrot-clean-valid\n```\n\nAnd then pass the paths to the datasets when we run the training script:\n\n```bash\naccelerate launch scripts/codeparrot_training.py \\n--model_ckpt codeparrot/codeparrot-small \\n--dataset_name_train ./data/codeparrot-clean-train \\n--dataset_name_valid ./data/codeparrot-clean-valid \\n--train_batch_size 12 \\n--valid_batch_size 12 \\n--learning_rate 5e-4 \\n--num_warmup_steps 2000 \\n--gradient_accumulation 1 \\n--gradient_checkpointing False \\n--max_train_steps 150000 \\n--save_checkpoint_steps 15000\n```",What is the name of the pre-trained model checkpoint used in the training script?,codeparrot/codeparrot-small,huggingface/transformers/blob/main/examples/research_projects/codeparrot/README.md
2,"Fortunately, you will need to do this only once because you can save your model and reload it later.\n\n```\n>>> model.save_pretrained(""a_local_path_for_compiled_neuron_model"")\n```\n\nEven better, you can push it to the [Hugging Face hub](https://huggingface.co/models).\n\n```\n>>> model.push_to_hub(\n ""a_local_path_for_compiled_neuron_model"",\n repository_id=""aws-neuron/Llama-2-7b-hf-neuron-latency"")\n```\n\n## Generate Text using Llama 2 on AWS Inferentia2\n\nOnce your model has been exported, you can generate text using the transformers library, as it has been described in [detail in this previous post](https://huggingface.co/blog/how-to-generate).\n\n```\n>>> from optimum.neuron import NeuronModelForCausalLM\n>>> from transformers import AutoTokenizer\n\n>>> model = NeuronModelForCausalLM.from_pretrained('aws-neuron/Llama-2-7b-hf-neuron-latency')\n>>> tokenizer = AutoTokenizer.from_pretrained(""aws-neuron/Llama-2-7b-hf-neuron-latency"")",How do you save a compiled Neuron model?,"You can save a compiled Neuron model using the `save_pretrained` method and specify a local path, or push it to the Hugging Face hub.",huggingface/blog/blob/main/inferentia-llama2.md
3,"Every time the pipeline is run, [`torch.randn`](https://pytorch.org/docs/stable/generated/torch.randn.html) uses a different random seed to create Gaussian noise which is denoised stepwise. This leads to a different result each time it is run, which is great for diffusion pipelines since it generates a different random image each time.\n\nBut if you need to reliably generate the same image, that'll depend on whether you're running the pipeline on a CPU or GPU.\n\n### CPU\n\nTo generate reproducible results on a CPU, you'll need to use a PyTorch [`Generator`](https://pytorch.org/docs/stable/generated/torch.Generator.html) and set a seed:\n\n```python\nimport torch\nfrom diffusers import

### **1.3. Setup critique agents**

The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practitioners.

One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___

We now build and run these critique agents.

In [17]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [19]:

import re

def parse_llm_judge_output(text):
    score_match = re.search(r"Total rating:\s*([1-5])", text, re.IGNORECASE)
    eval_match = re.search(
        r"Evaluation:\s*(.*?)(?:\n\s*Total rating:|$)",
        text,
        re.IGNORECASE | re.DOTALL
    )

    if not score_match or not eval_match:
        raise ValueError("Cannot parse judge output")

    score = int(score_match.group(1))
    evaluation = eval_match.group(1).strip()

    return score, evaluation


In [29]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
      for criterion, evaluation in evaluations.items():
        score, eval_text = parse_llm_judge_output(evaluation)

        output.update({
            f"{criterion}_score": score,
            f"{criterion}_eval": eval_text,
        })
    except Exception:
      continue


Generating critique for each QA couple...


  0%|          | 0/4 [00:00<?, ?it/s]

Now let us filter out bad questions based on our critique agent scores:

In [30]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What are the three major issues tackled in the training and application of large vision models?,"Training instability, resolution gaps between pre-training and fine-tuning, and hunger on labelled data.",5,4,5
1,What is the name of the pre-trained model checkpoint used in the training script?,codeparrot/codeparrot-small,4,4,4
2,How do you save a compiled Neuron model?,"You can save a compiled Neuron model using the `save_pretrained` method and specify a local path, or push it to the Hugging Face hub.",5,4,1
3,How can you generate reproducible results with a CPU?,You can generate reproducible results with a CPU by using a PyTorch Generator and setting a seed.,4,4,5


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What are the three major issues tackled in the training and application of large vision models?,"Training instability, resolution gaps between pre-training and fine-tuning, and hunger on labelled data.",5,4,5
1,What is the name of the pre-trained model checkpoint used in the training script?,codeparrot/codeparrot-small,4,4,4
3,How can you generate reproducible results with a CPU?,You can generate reproducible results with a CPU by using a PyTorch Generator and setting a seed.,4,4,5


Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

We have generated only a few QA couples here to reduce time and cost. But let's kickstart the next part by loading a pre-generated dataset:

Giờ thì bộ dữ liệu đánh giá tổng hợp của chúng ta đã hoàn chỉnh! Chúng ta có thể đánh giá các hệ thống **RAG khác nhau** trên bộ dữ liệu đánh giá này.

Chúng ta chỉ tạo ra một vài cặp QA ở đây để giảm thời gian và chi phí. Nhưng hãy bắt đầu phần tiếp theo bằng cách tải một bộ dữ liệu đã được tạo sẵn:

In [31]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

# **2. Build our RAG System**

### **2.1. Preprocessing documents to build our vector database**

- In this part, __we split the documents from our knowledge base into smaller chunks__: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

Many options exist for text splitting:
- split every `n` words / characters, but this has the risk of cutting in half paragraphs or even sentences
- split after `n` words / character, but only on sentence boundaries
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.

[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

> In the following, we use Langchain's `RecursiveCharacterTextSplitter`.

💡 _To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better._

In [32]:
# from langchain.docstore.document import Document as LangchainDocument
from langchain_core.documents import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = []

for doc in ds:
    lang_doc = LangchainDocument(
        page_content=doc["text"],
        metadata={"source": doc["source"]}
    )
    RAW_KNOWLEDGE_BASE.append(lang_doc)


In [33]:
#Check
print(len(RAW_KNOWLEDGE_BASE))
print(type(RAW_KNOWLEDGE_BASE[0]))
print(RAW_KNOWLEDGE_BASE[0].metadata)


2647
<class 'langchain_core.documents.base.Document'>
{'source': 'huggingface/hf-endpoints-documentation/blob/main/docs/source/guides/create_endpoint.mdx'}


In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### **2.2. Retriever - embeddings 🗂️**
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [56]:
from langchain_community.vectorstores import FAISS # FAISS = vector database
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os

def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },
    )


    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    # Check if embeddings already exist on disk
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )

        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

In [37]:
RAG_PROMPT_TEMPLATE = """[INST]
You are a helpful assistant.
Use ONLY the information in the context to answer the question.
If the answer cannot be deduced from the context, say "I don't know".

Context:
{context}

Question:
{question}
[/INST]
"""


In [58]:

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from google.colab import userdata
HF_API_TOKEN  = userdata.get('HF_KEY')


# Base LLM (text-generation)
base_llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    huggingfacehub_api_token=HF_API_TOKEN,
    max_new_tokens=512,
    temperature=0.1,
    repetition_penalty=1.03,
)

# Wrap --> Chat Model
READER_LLM = ChatHuggingFace(llm=base_llm)
repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = repo_id.split("/")[-1]

In [43]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Using the information contained in the context, "
     "give a comprehensive answer to the question. "
     "Respond only to the question asked. "
     "If the answer cannot be deduced from the context, say you don't know."),
    ("human",
     "Context:\n{context}\n\nQuestion: {question}")
])


In [44]:
def answer_with_rag(
    question: str,
    llm,
    knowledge_index,
    reranker=None,
    num_retrieved_docs=30,
    num_docs_final=7,
):
    # Retrieve
    docs = knowledge_index.similarity_search(question, k=num_retrieved_docs)
    docs = [d.page_content for d in docs]

    # Rerank (optional)
    if reranker:
        docs = reranker.rerank(question, docs, k=num_docs_final)
        docs = [d["content"] for d in docs]

    docs = docs[:num_docs_final]

    context = "\n\n".join(
        [f"Document {i}:\n{doc}" for i, doc in enumerate(docs)]
    )

    # Build chat prompt
    messages = rag_prompt.format_messages(
        question=question,
        context=context
    )

    # [
    # SystemMessage(content="Using the information..."),
    # HumanMessage(content="Context: ... Question: ...")
    # ]




    # Invoke chat model
    response = llm.invoke(messages)

    return response.content, docs


# **3. Benchmarking the RAG system**

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evaluation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on Answer Correctness since it is the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [45]:
print(len(eval_dataset))

65


In [49]:
from langchain_core.vectorstores import VectorStore
from ragatouille import RAGPretrainedModel
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

/tmp/ipython-input-1162113751.py:2: UserWarning: 
********************************************************************************
RAGatouille WARNING: Future Release Notice
--------------------------------------------
RAGatouille version 0.0.10 will be migrating to a PyLate backend 
instead of the current Stanford ColBERT backend.
PyLate is a fully mature, feature-equivalent backend, that greatly facilitates compatibility.
However, please pin version <0.0.10 if you require the Stanford ColBERT backend.
********************************************************************************
  from ragatouille import RAGPretrainedModel


In [50]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate



evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [51]:
from langchain_openai import ChatOpenAI
from google.colab import userdata
OPENAI_API_KEY =  userdata.get('GPT_KEY')

eval_chat_model = ChatOpenAI(model="gpt-4.1", temperature=0, openai_api_key=OPENAI_API_KEY, base_url="https://llm.ptnglobalcorp.com")
evaluator_name = "GPT4.1"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))
    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 Let's run the tests and evaluate answers!👇

In [59]:
# ===== CELL 1: RUN RAG ONLY =====

if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:
    for embeddings in ["thenlper/gte-small"]:
        for rerank in [True, False]:

            settings_name = (
                f"chunk:{chunk_size}_"
                f"embeddings:{embeddings.replace('/', '~')}_"
                f"rerank:{rerank}_"
                f"reader-model:{READER_MODEL_NAME}"
            )

            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running RAG for {settings_name}")

            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank else None
            )

            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )


Running RAG for chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta


/usr/local/lib/python3.12/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


  0%|          | 0/65 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()

100%|██████████| 1/1 [00:00<00:00,  9.45it/s]

100%|██████████| 1/1 [00:00<00:00,  9.49it/s]

100%|██████████| 1/1 [00:00<00:00,  9.34it/s]

100%|██████████| 1/1 [00:00<00:00,  9.40it/s]

100%|██████████| 1/1 [00:00<00:00,  9.45it/s]

100%|██████████| 1/1 [00:00<00:00,  9.20it/s]

100%|██████████| 1/1 [00:00<00:00,  9.05it/s]

100%|██████████| 1/1 [00:00<00:00, 11.01it/s]

100%|██████████| 1/1 [00:00<00:00,  9.38it/s]

100%|██████████| 1/1 [00:00<00:00,  9.22it/s]

100%|██████████| 1/1 [00:00<00:00,  9.43it/s]

100%|██████████| 1/1 [00:00<00:00,  9.34it/s]

100%|██████████| 1/1 [00:00<00:00,  9.32it/s]

100%|██████████| 1/1 [00:00<00:00,  9.34it/s]

100%|██████████| 1/1 [00:00<00:00,  9.28it/s]

100%|██████████| 1/1 [00:00

Running RAG for chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:zephyr-7b-beta


  0%|          | 0/65 [00:00<?, ?it/s]

In [60]:
# ===== CELL 2: RUN EVALUATION ONLY =====

for chunk_size in [200]:
    for embeddings in ["thenlper/gte-small"]:
        for rerank in [True, False]:

            settings_name = (
                f"chunk:{chunk_size}_"
                f"embeddings:{embeddings.replace('/', '~')}_"
                f"rerank:{rerank}_"
                f"reader-model:{READER_MODEL_NAME}"
            )

            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Evaluating results for {settings_name}")

            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )


Evaluating results for chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluating results for chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:zephyr-7b-beta


  0%|          | 0/65 [00:00<?, ?it/s]

### Inspect results

In [61]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [62]:
result["eval_score_GPT4.1"] = result["eval_score_GPT4.1"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4.1"] = (result["eval_score_GPT4.1"] - 1) / 4

In [64]:
average_scores = result.groupby("settings")["eval_score_GPT4.1"].mean()
average_scores.sort_values()

,eval_score_GPT4.1
settings,
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:zephyr-7b-beta.json,0.696154
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta.json,0.711538


## Example results

Let us load the results that I obtained by tweaking the different options available in this notebook.
For more detail on why these options could work or not, see the notebook on [advanced_RAG](advanced_rag).

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ ___There is no single good recipe: you should try several different directions when tuning your RAG systems.___


In [65]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

README.md:   0%|          | 0.00/319 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [66]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_settings_accuracy.png" height="500" width="800">

As you can see, these had varying impact on performance. In particular, tuning the chunk size is both easy and very impactful.

But this is our case: your results could be very different: now that you have a robust evaluation pipeline, you can set on to explore other options! 🗺️